In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from linearmodels import PanelOLS

In [166]:
data = pd.read_csv('data.csv',  usecols=range(10))
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d %H:%M:%S')
data.columns=['code','date','code_return','mkt','ten_return','smg','hml','umd','rmw','cma']
data

,code,date,code_return,mkt,ten_return,smg,hml,umd,rmw,cma
0,1,2010-01-05,-0.0173,0.006584,-0.0062,0.005921,-0.002548,-0.001689,0.001728,0.000770
1,2142,2010-01-05,0.0024,0.006584,-0.0062,0.005921,-0.002548,-0.001689,0.001728,0.000770
2,600000,2010-01-05,0.0076,0.006584,-0.0062,0.005921,-0.002548,-0.001689,0.001728,0.000770
3,600015,2010-01-05,0.0252,0.006584,-0.0062,0.005921,-0.002548,-0.001689,0.001728,0.000770
4,600016,2010-01-05,0.0177,0.006584,-0.0062,0.005921,-0.002548,-0.001689,0.001728,0.000770
...,...,...,...,...,...,...,...,...,...,...
46006,601398,2021-12-31,-0.0022,0.005888,-0.0086,-0.004844,0.003232,0.002873,-0.004371,0.004678
46007,601818,2021-12-31,0.0000,0.005888,-0.0086,-0.004844,0.003232,0.002873,-0.004371,0.004678
46008,601939,2021-12-31,-0.0017,0.005888,-0.0086,-0.004844,0.003232,0.002873,-0.004371,0.004678
46009,601988,2021-12-31,0.0000,0.005888,-0.0086,-0.004844,0.003232,0.002873,-0.004371,0.004678


In [167]:
#将年份分组
print(np.unique(data['code']))
grouped_df = {str(year): data.loc[data['date'].dt.year == year] for year in data['date'].dt.year.unique()}
grouped_df

[     1   2142 600000 600015 600016 600036 601009 601166 601169 601288
 601328 601398 601818 601939 601988 601998]


{'2010':         code       date  code_return       mkt  ten_return       smg  \
 0          1 2010-01-05      -0.0173  0.006584     -0.0062  0.005921   
 1       2142 2010-01-05       0.0024  0.006584     -0.0062  0.005921   
 2     600000 2010-01-05       0.0076  0.006584     -0.0062  0.005921   
 3     600015 2010-01-05       0.0252  0.006584     -0.0062  0.005921   
 4     600016 2010-01-05       0.0177  0.006584     -0.0062  0.005921   
 ...      ...        ...          ...       ...         ...       ...   
 3423  601398 2010-12-31       0.0000  0.021355      0.2259 -0.001975   
 3424  601818 2010-12-31       0.0128  0.021355      0.2259 -0.001975   
 3425  601939 2010-12-31       0.0066  0.021355      0.2259 -0.001975   
 3426  601988 2010-12-31       0.0031  0.021355      0.2259 -0.001975   
 3427  601998 2010-12-31       0.0096  0.021355      0.2259 -0.001975   
 
            hml       umd       rmw      cma  
 0    -0.002548 -0.001689  0.001728  0.00077  
 1    -0.002548 -0.0

In [168]:
unique_stocks=data['code'].unique()
return_board=pd.DataFrame(index=range(len(unique_stocks)),columns=range(len(grouped_df)))
return_board_t=pd.DataFrame(index=range(len(unique_stocks)),columns=range(len(grouped_df)))
return_board_p=pd.DataFrame(index=range(len(unique_stocks)),columns=range(len(grouped_df)))
coef=[]
for i,(year,dt) in enumerate(grouped_df.items()):
    for j,stock in enumerate(unique_stocks):
        stock_data=dt[dt['code']==stock]
        X=stock_data[['mkt','ten_return','smg','hml','umd','rmw','cma']]
        X=sm.add_constant(X)
        y=stock_data[['code_return']]
        model= sm.OLS(y,X).fit()

        nobs, _ = X.shape
        nw_lags = round(pow(nobs,0.25))
        nw_model = model.get_robustcov_results(cov_type='HAC', maxlags=nw_lags)
        return_board.iloc[j, i] = nw_model.params[2]
        return_board_t.iloc[j,i]=nw_model.tvalues[2]
        return_board_p.iloc[j,i]=nw_model.pvalues[2]


return_board

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.013148,0.005773,-0.002174,-0.011488,-0.003769,-0.008502,0.000224,-0.005029,0.006808,-0.007678,-0.00327,-0.041562
1,0.00311,0.006051,-0.000892,-0.006017,0.00272,-0.002792,0.006169,0.00631,0.002925,-0.007352,0.008641,-0.038542
2,0.00621,0.006622,-0.001221,-0.000178,0.002632,0.000758,0.00916,0.007962,0.008392,-0.001293,-0.004359,-0.010352
3,0.01093,0.011443,-0.003068,-0.004101,0.001881,0.001777,0.005978,0.001063,0.004835,-0.002413,0.000799,-0.004831
4,0.003502,0.004323,-0.005825,0.000091,0.00042,-0.002892,-0.000686,0.00123,0.002731,-0.004258,-0.002813,-0.006835
5,0.008415,0.007548,-0.005569,-0.00101,-0.002053,-0.004033,0.001196,0.000351,-0.003022,0.00475,-0.00051,-0.028943
6,0.011578,0.005271,-0.005497,-0.001595,-0.000909,-0.006777,0.00702,0.000757,-0.004006,-0.013887,0.000157,-0.032707
7,0.012647,0.007237,-0.004455,-0.006842,-0.000847,-0.004233,0.002475,0.004029,0.002705,-0.006041,-0.000705,-0.034017
8,0.014949,0.006399,-0.006739,-0.003964,-0.001614,0.001471,-0.004751,-0.000544,0.001677,-0.004262,-0.001848,-0.002259
9,0.007986,0.002128,-0.003844,-0.004521,0.003023,-0.005343,0.006551,0.004729,0.000891,0.001353,-0.004274,-0.005001


In [169]:
return_board_t

,0,1,2,3,4,5,6,7,8,9,10,11
0,2.672289,1.65195,-0.584873,-1.422226,-0.679915,-1.713695,0.08846,-0.887568,0.642572,-0.747215,-0.552356,-2.734635
1,0.678286,1.692203,-0.176714,-1.473519,0.581675,-0.359514,0.837783,0.949781,0.315842,-0.874458,1.209169,-3.172067
2,1.607803,2.091248,-0.338955,-0.033024,0.498631,0.133049,1.979966,1.892292,1.470652,-0.186103,-1.553808,-2.368739
3,1.940856,3.025431,-0.648008,-0.946093,0.335058,0.34687,0.834401,0.293936,1.262213,-0.562626,0.281094,-1.246859
4,1.085603,1.450319,-1.33474,0.016554,0.066288,-0.531891,-0.124888,0.316434,0.630516,-1.259602,-1.052676,-2.037289
5,1.598023,2.408994,-1.237503,-0.186233,-0.47784,-0.775524,0.22234,0.064866,-0.333926,0.667069,-0.072881,-2.340295
6,2.263565,2.002715,-1.12967,-0.485346,-0.149489,-0.935721,0.913949,0.124861,-0.562,-1.711675,0.039256,-3.033372
7,2.510193,2.194232,-1.085532,-1.116724,-0.167024,-0.657669,0.53287,1.338756,0.683224,-0.783068,-0.160481,-2.897619
8,3.156239,2.608247,-1.773021,-0.792807,-0.300002,0.227941,-0.970723,-0.134426,0.391561,-0.843345,-0.826849,-0.660071
9,1.908888,1.020874,-1.252643,-1.284519,0.497454,-0.836817,1.27097,1.555012,0.203586,0.223907,-1.930875,-1.255167


In [170]:
return_board_p

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.008214,0.099921,0.559219,0.156321,0.497225,0.087898,0.929586,0.375677,0.521128,0.455678,0.58123,0.006721
1,0.498269,0.09195,0.859886,0.141979,0.561341,0.719532,0.403004,0.343195,0.752403,0.382758,0.227813,0.001721
2,0.109306,0.03758,0.734947,0.973684,0.618504,0.894275,0.048965,0.059676,0.142723,0.852524,0.121576,0.018659
3,0.053577,0.002759,0.517615,0.345094,0.737878,0.728998,0.404899,0.769065,0.208123,0.574223,0.778885,0.213691
4,0.278781,0.14832,0.183275,0.986807,0.947204,0.595302,0.900719,0.751953,0.52897,0.209057,0.293571,0.042742
5,0.111435,0.01677,0.217141,0.85243,0.633205,0.438823,0.824242,0.948336,0.738733,0.505379,0.941962,0.020106
6,0.024559,0.046357,0.259772,0.627893,0.881296,0.350394,0.361676,0.900741,0.574652,0.08827,0.96872,0.00269
7,0.01277,0.029198,0.278827,0.265277,0.867493,0.511392,0.594636,0.181938,0.495138,0.434372,0.87264,0.004115
8,0.001811,0.009692,0.077554,0.428707,0.76444,0.819919,0.33268,0.893184,0.695738,0.399889,0.409161,0.509855
9,0.057549,0.308368,0.211597,0.200253,0.61933,0.403542,0.204991,0.121283,0.838853,0.823023,0.054701,0.210665


In [171]:
data2 = pd.read_csv('data2.csv',  usecols=range(4))
data2.columns=['date','code','gap','price']
data2['date'] = pd.to_datetime(data2['date'], format='%Y')
data2 = data2.sort_values(by=['code','date'])
data2

,date,code,gap,price
0,2010-01-01,1,-6.894123e+10,5.502837e+10
1,2011-01-01,1,9.845070e+08,7.987303e+10
2,2012-01-01,1,-1.279340e+11,8.207607e+10
3,2013-01-01,1,-8.280000e+10,1.000000e+11
4,2014-01-01,1,-9.816000e+10,1.810000e+11
...,...,...,...,...
139,2017-01-01,601998,-1.191440e+12,7.367929e+10
140,2018-01-01,601998,-1.148510e+12,6.111160e+10
141,2019-01-01,601998,-1.075820e+12,6.936214e+10
142,2020-01-01,601998,-7.523680e+11,6.024817e+10


In [172]:
p1=pd.DataFrame(index=range(len(unique_stocks)),columns=range(len(grouped_df)))
p2=pd.DataFrame(index=range(len(unique_stocks)),columns=range(len(grouped_df)))
unique_stocks=data2['code'].unique()
unique_years=data2['date'].unique()
def to_panel(x):
    global p1,p2
    p1[np.where(unique_years == x[0])[0][0]][np.where(unique_stocks == x[1])[0][0]] =x[2]
    p2[np.where(unique_years == x[0])[0][0]][np.where(unique_stocks == x[1])[0][0]] =x[3]
data2.apply(to_panel,axis=1)
p1

,0,1,2,3,4,5,6,7,8,9,10,11
0,-68941229000.0,984507000.0,-127934000000.0,-82800000000.0,-98160000000.0,-157673000000.0,-508704000000.0,-315695000000.0,-190340000000.0,-556431000000.0,-762220000000.0,-824255000000.0
1,-23688370000.0,-30949262000.0,-64387527000.0,-96688319000.0,-91045918000.0,-285293000000.0,-299220000000.0,-451783000000.0,-420995000000.0,-443250000000.0,-631523000000.0,-731100000000.0
2,-141328000000.0,-122787000000.0,-262943000000.0,-373384000000.0,-470193000000.0,-984973000000.0,-766507000000.0,-981123000000.0,-616548000000.0,-817397000000.0,-2091120000000.0,-1822820000000.0
3,-28462620000.0,-20438980000.0,-126522000000.0,5316000000.0,82059000000.0,-328526000000.0,-324292000000.0,-289730000000.0,119941000000.0,-47074000000.0,-466522000000.0,-370309000000.0
4,271833000000.0,2799100000000.0,3916390000000.0,412611000000.0,256403000000.0,217277000000.0,-63557000000.0,-701791000000.0,-325998000000.0,-510123000000.0,-1331160000000.0,1124930000000.0
5,-208027000000.0,-147414000000.0,-355760000000.0,-547428000000.0,-492849000000.0,-313807000000.0,-772892000000.0,-1394600000000.0,-1358110000000.0,-1521390000000.0,-2165980000000.0,-2331830000000.0
6,-63128641000.0,-79853308000.0,-86416577000.0,-88916343000.0,-117734000000.0,-128597000000.0,-211613000000.0,-208129000000.0,-30603972000.0,88444896000.0,-158145000000.0,-139295000000.0
7,-92230000000.0,-150749000000.0,-543959000000.0,-580710000000.0,-607199000000.0,-1168380000000.0,-1352810000000.0,-984578000000.0,-352581000000.0,112035000000.0,-457532000000.0,-946785000000.0
8,63789908000.0,12461318000.0,-41682796000.0,-132729000000.0,-71886000000.0,-158707000000.0,-197487000000.0,-181977000000.0,-57826000000.0,-132071000000.0,-266780000000.0,-162072000000.0
9,-1691760000000.0,-1557840000000.0,-1565400000000.0,-1743650000000.0,-1832790000000.0,-2413880000000.0,-3214940000000.0,-2985350000000.0,-3008970000000.0,-3639710000000.0,-7179610000000.0,-5716900000000.0


In [173]:
p2

,0,1,2,3,4,5,6,7,8,9,10,11
0,55028367302.0,79873032985.0,82076073664.0,100000000000.0,181000000000.0,172000000000.0,156000000000.0,228000000000.0,161000000000.0,319000000000.0,375000000000.0,320000000000.0
1,35759374560.0,26415796046.0,30741526839.0,26617663483.0,51119800748.0,60485806196.0,64892573508.0,90291932352.0,84482768312.0,158000000000.0,212000000000.0,253000000000.0
2,178000000000.0,158000000000.0,185000000000.0,176000000000.0,293000000000.0,341000000000.0,350000000000.0,370000000000.0,288000000000.0,363000000000.0,284000000000.0,250000000000.0
3,226000000000.0,210000000000.0,243000000000.0,225000000000.0,342000000000.0,371000000000.0,363000000000.0,599000000000.0,520000000000.0,775000000000.0,907000000000.0,1000000000000.0
4,140000000000.0,129000000000.0,137000000000.0,123000000000.0,260000000000.0,230000000000.0,218000000000.0,211000000000.0,186000000000.0,210000000000.0,174000000000.0,157000000000.0
5,163000000000.0,147000000000.0,194000000000.0,151000000000.0,267000000000.0,253000000000.0,226000000000.0,244000000000.0,227000000000.0,221000000000.0,176000000000.0,181000000000.0
6,632000000000.0,571000000000.0,571000000000.0,513000000000.0,851000000000.0,845000000000.0,725000000000.0,837000000000.0,761000000000.0,778000000000.0,670000000000.0,643000000000.0
7,113000000000.0,133000000000.0,178000000000.0,174000000000.0,296000000000.0,285000000000.0,268000000000.0,248000000000.0,203000000000.0,224000000000.0,184000000000.0,138000000000.0
8,54396758644.0,76922420464.0,51651968071.0,76312794872.0,120000000000.0,130000000000.0,116000000000.0,115000000000.0,94759654366.0,118000000000.0,96170149894.0,86168454305.0
9,788000000000.0,770000000000.0,823000000000.0,729000000000.0,1090000000000.0,950000000000.0,912000000000.0,1130000000000.0,1150000000000.0,1180000000000.0,1000000000000.0,939000000000.0


In [174]:
beta_1= p1.div(p2)
beta_1=beta_1.drop(beta_1.columns[0],axis=1)
beta_1.rename(columns=lambda x:int(x-1),inplace=True)
beta_1

,0,1,2,3,4,5,6,7,8,9,10
0,0.012326,-1.558725,-0.828,-0.54232,-0.916703,-3.260923,-1.384627,-1.182236,-1.744298,-2.032587,-2.575797
1,-1.17162,-2.09448,-3.632487,-1.78103,-4.716693,-4.611005,-5.003581,-4.983206,-2.80538,-2.978882,-2.889723
2,-0.777133,-1.421314,-2.1215,-1.604754,-2.888484,-2.19002,-2.651684,-2.140792,-2.251782,-7.363099,-7.29128
3,-0.097328,-0.520667,0.023627,0.239939,-0.885515,-0.893366,-0.483689,0.230656,-0.060741,-0.514357,-0.370309
4,21.69845,28.586788,3.354561,0.986165,0.944683,-0.291546,-3.326024,-1.752677,-2.429157,-7.650345,7.165159
5,-1.002816,-1.833814,-3.625351,-1.845876,-1.240344,-3.419876,-5.715574,-5.982863,-6.884118,-12.306705,-12.883039
6,-0.139848,-0.151343,-0.173326,-0.138348,-0.152186,-0.29188,-0.248661,-0.040215,0.113682,-0.236037,-0.216633
7,-1.133451,-3.055949,-3.337414,-2.051348,-4.099579,-5.047799,-3.970073,-1.736852,0.500156,-2.486587,-6.860761
8,0.161999,-0.806993,-1.739276,-0.59905,-1.220823,-1.702474,-1.582409,-0.610239,-1.119246,-2.774042,-1.880874
9,-2.023169,-1.902066,-2.391838,-1.681459,-2.540926,-3.525154,-2.641903,-2.616496,-3.0845,-7.17961,-6.088285


In [175]:
beta_2=pd.DataFrame(index=range(len(unique_stocks)),columns=range(len(grouped_df)-1))
for n in range(11):
    beta_2[n]=p1.iloc[:,n].div(p2.iloc[:,n+1])
beta_2

,0,1,2,3,4,5,6,7,8,9,10
0,-0.863135,0.011995,-1.27934,-0.457459,-0.570698,-1.010724,-2.231158,-1.960839,-0.596677,-1.483816,-2.381937
1,-0.89675,-1.006757,-2.418977,-1.891406,-1.505244,-4.396389,-3.313917,-5.347635,-2.664525,-2.090802,-2.496138
2,-0.894481,-0.663714,-1.493994,-1.274348,-1.378865,-2.814209,-2.071641,-3.406677,-1.698479,-2.878158,-8.36448
3,-0.135536,-0.084111,-0.56232,0.015544,0.221183,-0.90503,-0.541389,-0.557173,0.154763,-0.051901,-0.466522
4,2.107233,20.431387,31.840569,1.586965,1.114796,0.996683,-0.301218,-3.77307,-1.552371,-2.931741,-8.478726
5,-1.41515,-0.759866,-2.356026,-2.050292,-1.94802,-1.388527,-3.16759,-6.143612,-6.145294,-8.644261,-11.96674
6,-0.110558,-0.139848,-0.168453,-0.104485,-0.13933,-0.177375,-0.252823,-0.273494,-0.039337,0.132007,-0.245949
7,-0.693459,-0.846904,-3.126201,-1.961858,-2.130523,-4.359627,-5.454879,-4.850138,-1.574022,0.608886,-3.315449
8,0.829276,0.241255,-0.54621,-1.106075,-0.552969,-1.368164,-1.717278,-1.920406,-0.490051,-1.373306,-3.096029
9,-2.197091,-1.89288,-2.147325,-1.599679,-1.929253,-2.646798,-2.84508,-2.595957,-2.549975,-3.63971,-7.646017


In [176]:
y_panel=return_board.drop(return_board.columns[0],axis=1)
y_panel.rename(columns=lambda x:int(x-1),inplace=True)
y_panel

,0,1,2,3,4,5,6,7,8,9,10
0,0.005773,-0.002174,-0.011488,-0.003769,-0.008502,0.000224,-0.005029,0.006808,-0.007678,-0.00327,-0.041562
1,0.006051,-0.000892,-0.006017,0.00272,-0.002792,0.006169,0.00631,0.002925,-0.007352,0.008641,-0.038542
2,0.006622,-0.001221,-0.000178,0.002632,0.000758,0.00916,0.007962,0.008392,-0.001293,-0.004359,-0.010352
3,0.011443,-0.003068,-0.004101,0.001881,0.001777,0.005978,0.001063,0.004835,-0.002413,0.000799,-0.004831
4,0.004323,-0.005825,0.000091,0.00042,-0.002892,-0.000686,0.00123,0.002731,-0.004258,-0.002813,-0.006835
5,0.007548,-0.005569,-0.00101,-0.002053,-0.004033,0.001196,0.000351,-0.003022,0.00475,-0.00051,-0.028943
6,0.005271,-0.005497,-0.001595,-0.000909,-0.006777,0.00702,0.000757,-0.004006,-0.013887,0.000157,-0.032707
7,0.007237,-0.004455,-0.006842,-0.000847,-0.004233,0.002475,0.004029,0.002705,-0.006041,-0.000705,-0.034017
8,0.006399,-0.006739,-0.003964,-0.001614,0.001471,-0.004751,-0.000544,0.001677,-0.004262,-0.001848,-0.002259
9,0.002128,-0.003844,-0.004521,0.003023,-0.005343,0.006551,0.004729,0.000891,0.001353,-0.004274,-0.005001


In [177]:
years = y_panel.columns
stocks = y_panel.index

# 创建多索引数据框
multi_index = pd.MultiIndex.from_product([stocks, years], names=['stock', 'year'])

# 初始化面板数据框
panel_data = pd.DataFrame(index=multi_index, columns=['y', 'p1', 'p2'])

# 填充面板数据框
for stock in stocks:
    for year in years:
        panel_data.loc[(stock, year), 'y'] = y_panel.loc[stock, year]
        panel_data.loc[(stock, year), 'p1'] = p1.loc[stock, year]
        panel_data.loc[(stock, year), 'p2'] = p2.loc[stock, year]
        for i in range(len(y_panel.index)-1):
            panel_data.loc[(stock,year),'dummy-stock-'+str(i)] = 1 if i==stock else 0
        for i in range(len(y_panel.columns)-1):
            panel_data.loc[(stock,year),'dummy-year-'+str(i)] = 1 if i==year else 0
# 转换为linearmodels库可识别的格式
panel_data = panel_data.astype(np.float64)
panel_data = panel_data.reset_index().set_index(['stock', 'year'])
panel_data

y            p1            p2  dummy-stock-0  \
stock year                                                        
0     0     0.005773 -6.894123e+10  5.502837e+10            1.0   
      1    -0.002174  9.845070e+08  7.987303e+10            1.0   
      2    -0.011488 -1.279340e+11  8.207607e+10            1.0   
      3    -0.003769 -8.280000e+10  1.000000e+11            1.0   
      4    -0.008502 -9.816000e+10  1.810000e+11            1.0   
...              ...           ...           ...            ...   
15    6     0.003498 -1.174080e+12  5.218950e+10            0.0   
      7    -0.000600 -1.191440e+12  7.367929e+10            0.0   
      8     0.001028 -1.148510e+12  6.111160e+10            0.0   
      9    -0.005841 -1.075820e+12  6.936214e+10            0.0   
      10   -0.009565 -7.523680e+11  6.024817e+10            0.0   

            dummy-stock-1  dummy-stock-2  dummy-stock-3  dummy-stock-4  \
stock year                                                               
0     0               0.0            0.0            0.0            0.0   
      1               0.0            0.0            0.0            0.0   
      2               0.0            0.0            0.0            0.0   
      3               0.0            0.0            0.0            0.0   
      4               0.0            0.0            0.0            0.0   
...                   ...            ...            ...            ...   
15    6               0.0            0.0            0.0            0.0   
      7               0.0            0.0            0.0            0.0   
      8               0.0            0.0            0.0            0.0   
      9               0.0            0.0            0.0            0.0   
      10              0.0            0.0            0.0            0.0   

            dummy-stock-5  dummy-stock-6  ...  dummy-year-0  dummy-year-1  \
stock year                                ...                               
0     0               0.0            0.0  ...           1.0           0.0   
      1               0.0            0.0  ...           0.0           1.0   
      2               0.0            0.0  ...           0.0           0.0   
      3               0.0            0.0  ...           0.0           0.0   
      4               0.0            0.0  ...           0.0           0.0   
...                   ...            ...  ...           ...           ...   
15    6               0.0            0.0  ...           0.0           0.0   
      7               0.0            0.0  ...           0.0           0.0   
      8               0.0            0.0  ...           0.0           0.0   
      9               0.0            0.0  ...           0.0           0.0   
      10              0.0            0.0  ...           0.0           0.0   

            dummy-year-2  dummy-year-3  dummy-year-4  dummy-year-5  \
stock year                                                           
0     0              0.0           0.0           0.0           0.0   
      1              0.0           0.0           0.0           0.0   
      2              1.0           0.0           0.0           0.0   
      3              0.0           1.0           0.0           0.0   
      4              0.0           0.0           1.0           0.0   
...                  ...           ...           ...           ...   
15    6              0.0           0.0           0.0           0.0   
      7              0.0           0.0           0.0           0.0   
      8              0.0           0.0           0.0           0.0   
      9              0.0           0.0           0.0           0.0   
      10             0.0           0.0           0.0           0.0   

            dummy-year-6  dummy-year-7  dummy-year-8  dummy-year-9  
stock year                                                          
0     0              0.0           0.0           0.0           0.0  
      1              0.0           0.0           0.0           0

In [ ]:
dummy1=['dummy-stock-'+str(i) for i in range(len(y_panel.index)-1)]
dummy2=['dummy-year-'+str(i) for i in range(len(y_panel.columns)-1)]

const_col = pd.DataFrame(np.ones((len(panel_data), 1)), index=panel_data.index, columns=['const'])
panel_data = pd.concat([const_col,panel_data], axis=1)

col_names = ['const','p1', 'p2'] + list(dummy1+dummy2)
mod = PanelOLS(panel_data.y, panel_data[col_names],check_rank=False)
res = mod.fit()

# 输出回归结果
print(res)